## Accessing and Visualizing Device Locations in Oceans 2.0

To motivate this work, from [Ocean Networks Canada](http://www.oceannetworks.ca) (ONC):

>"Long-term, continuous scientific data from the ocean environment are gathered by Ocean Networks Canada and made available through **Oceans 2.0**—a powerful online data management system. Oceans 2.0, combined with high-performance computing, allows ONC to provide ocean analytics that assist researchers, communities, industry, and policy-makers in making evidence-based decisions in Canada and globally."

The API documentation [here](https://wiki.oceannetworks.ca/display/O2A/API+Reference) captures ONC's mission with Oceans 2.0, but allow us to paraphrase.

What we have is a public archive of oceans data that is programmatically accessible. This data is served in three forms:

* **Discovery**:  we need identifier codes to find devices.
* **Data Delivery**: once we have found a device, we want its data!
* **Interoperability**: we may want to share with or integrate other data sources.

We could, of course, access the same data [here](https://data.oceannetworks.ca/), but our aim is leverage Jupyter in place of a _fixed_ interface.

---

Let's look at **Discovery**.

We will refine our search, filtering by one of `locations`, `devices`, `deviceCategories`, `properties`, `dataProducts`, and `deployments`.

Our data journey begins with `locations` $\dots$

> "Find locations that have the data you are interested in and use the locationCode when requesting a data using the `dataProductDelivery`, `scalardata` or `rawdata` web services."

First and foremost, we need to make a request to $\text{Oceans} \ 2.0$.

Python offers a convenient library for doing this. Enter `requests`.

In [ ]:
import requests

Here `requests` expects a valid base `url` to which we will pass search parameters.

Since we seek `locations`, we append this to the `url`. 

In [ ]:
url = 'https://data.oceannetworks.ca/api/'
searchFilter = 'locations'
url += searchFilter
url

We wish to "`get`" data, so that's our `method` parameter.

Also, ONC expects an access token to authenticate the request, so we include one of those, too.

The access token is located in `Oceans2ApiToken.txt`, within our current directory.

In [ ]:
with open('Oceans2ApiToken.txt', 'r') as f:
    parameters = {
        'method':'get',
        'token': f.read()
    }

Now `requests` _also_ needs to be told that we are "getting" data.

Upon initializing `response`, we make a request to Oceans 2.0.

In [ ]:
response = requests.get(url, params=parameters)

We may check that the `response` is well-received.

In [ ]:
response.ok

$\downarrow \ $ We should also be reluctant to move on if this status code is anything _but_ $200$.

In [ ]:
response.status_code

If all is well,  we proceed.

---

The data we receive is formatted as [$\text{JSON}$](https://www.json.org/). This needs to be parsed for Python to handle it elegantly, so we import another library.

In [ ]:
import json

The `content` of our `response` is where the data lingers.

To keep the formatting consistent, we convert our newfound `data` to a `UTF-8` `str`ing.

Then, we use  `json.loads` to yield a `list`.

In [ ]:
jsonData = str(response.content, 'utf-8')
data = json.loads(jsonData)
type(jsonData) # just checking ...

We may [slice](https://en.wikipedia.org/wiki/Array_slicing) the first $3$ elements of our `data` for a preview.

$\downarrow \ $ We anticipate a list of dictionaries, where each dictionary corresponds to a location.

In [ ]:
data[:3]

Above we should see some coordinates, perhaps a description, etc.

The `deployments`, `hasDeviceData` and `hasPropertyData` keys are valuable. These tell us if and how many devices there are somewhere. What is more, they tell us if we can "observe" the ocean from there.

---

Let's transform our newfound data into a `DataFrame` via [`pandas`](https://pandas.pydata.org/).

In [ ]:
import pandas

In [ ]:
dataFrame = pandas.read_json(jsonData)
dataFrame

There is quite a bit of information here. We may get the "length," i.e. the number of locations, in our `dataFrame`.

In [ ]:
len(dataFrame)

Also, there is a programmatic way to see what data is visible for each location.

In [ ]:
dataFrame.columns.values

For now, to see where the data is coming from, let's grab the coordinates of these locations.

Again we slice the first $3$ elements of our `latitudes` and `longitudes` for a preview.

In [ ]:
latitudes = dataFrame['lat']
latitudes[:3]

In [ ]:
longitudes = dataFrame['lon']
longitudes[:3]

It seems that some locations do not have registered coordinates.

Since these locations are not immediately informative, we will exclude them and [`zip`](https://en.wikipedia.org/wiki/Convolution_(computer_science) the valid `latitudes` with their corresponding `longitudes`.

A subtle way to catch and exclude `NaNs` in a DataFrames is to check if an element equals itself. To be clear, `NaN != NaN`.

We see this below as `l` ... `if l == l`. Elements that fail to meet this criteria are excluded.

In [ ]:
coordinates = list(zip(
    [l for l in latitudes if l == l],
    [l for l in longitudes if l == l]
))

Once more we slice some elements of `coordinates` to see that the structure meets our expectations.

In [ ]:
coordinates[:3]

---

We may visualize our `coordinates` with [$\mathbf{D3}$](https://d3js.org/)$.js,$ but we still need to pass the `coordinates` to our current window via $\text{JavaScript}.$

For now, we cheat by globally initializing them.

In [ ]:
from IPython.display import Javascript
Javascript("window.coordinates = %s" % json.dumps({'coordinates': coordinates}))

We have also blackboxed the code needed to generate an interactive map using a [Mercator projection](https://en.wikipedia.org/wiki/Mercator_projection).

If we're curious, we may look in our current directory at `d3Geography.py` to see what's been abstracted.

The file `americas.geojson`, which is also in our current directory, contains the geographic data.

In [ ]:
from d3Geography import SVG, geography
view = geography("americas.geojson")
svg = SVG(height=500)

view.addProjection("Mercator")
view.addBrush() # drag to select a region
view.addZoom()  # zoom to region
view.make(svg=svg)

$\uparrow \ $ Look above! The <font color="red">red</font> circles are the `coordinates` in our `data`.

We may _click_ on these circles to zoom towards one, or _drag_ to select and zoom into a specific region.

Also, double-clicking the white space (i.e. the ocean) at any time will restore the initial view. 

If any circles seem out of place, try running the previous cell once more to refresh things.

---

* _Assembled by Eric Easthope_

* I humbly and _**heavily**_ borrow bits from [bl.ocks](https://bl.ocks.org/) and [Stack Overflow](https://stackoverflow.com/).

---